In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from mingptf.model import GPT
import numpy as np
import tensorflow as tf
from mingptf.bpe import BPETokenizer
from transformers import GPT2Tokenizer, GPT2LMHeadModel

2023-01-09 21:27:43.990802: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/abhaykumar/opt/anaconda3/envs/mingpt/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
use_mingpt = True # use minGPT or huggingface/transformers model?
model_type = 'gpt2'

In [4]:
if use_mingpt:
    model = GPT.from_pretrained(model_type)
else:
    model = GPT2LMHeadModel.from_pretrained(model_type)
    model.config.pad_token_id = model.config.eos_token_id # suppress a warning


2023-01-09 21:27:52.104309: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


number of parameters: 124.44M


All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [21]:
tokenizer = BPETokenizer()
hf_tokenizer = GPT2Tokenizer.from_pretrained(model_type)

def generate(prompt='', num_samples=10, no_gen_tokens=20, no_samples=5, temperature=1, top_k=8):
        
    # tokenize the input prompt into integer input sequence
    if use_mingpt:
        if prompt == '':
            # to create unconditional samples...
            # manually create a tensor with only the special <|endoftext|> token
            # similar to what openai's code does here https://github.com/openai/gpt-2/blob/master/src/generate_unconditional_samples.py
            x = [[tokenizer.encoder.encoder['<|endoftext|>']]]
        else:
            x = tokenizer(prompt)
    else:
        if prompt == '': 
            # to create unconditional samples...
            # huggingface/transformers tokenizer special cases these strings
            prompt = '<|endoftext|>'
        encoded_input = hf_tokenizer(prompt, return_tensors='tf')
        x = encoded_input['input_ids']
    
    # we'll process all desired num_samples in a batch, so expand out the batch dim

    # forward the model `steps` times to get samples, in a batch
    samples = []
    for _ in range(no_samples):
        y = model.generate(x, max_new_tokens=no_gen_tokens, 
                           temperature=temperature,
                           top_k=top_k)
        samples.append(tokenizer.decode(y.numpy()))
    
    return samples
        

In [22]:
gen = generate(prompt='Elon Musk is a', 
         no_gen_tokens=20,
         no_samples=5,
         temperature=1.2, 
         top_k=8)

for generated_seq in gen:
    print('-'*60)
    print(generated_seq)

------------------------------------------------------------
Elon Musk is a former president of the National Association of Realtors and CEO of SpaceX's parent company, Elon Musk
------------------------------------------------------------
Elon Musk is a big believer in the power of the blockchain. He has been one of the most vocal in pushing for
------------------------------------------------------------
Elon Musk is a professor of computer science and artificial intelligence at Harvard University and a member of the National Academy of Sciences Advisory
------------------------------------------------------------
Elon Musk is a former chief operating officer for Microsoft. He is the co-director of Microsoft Research, who has been
------------------------------------------------------------
Elon Musk is a member of Elon Musk's team and is the chief executive of SolarCity, which owns Tesla Motors Co
